## Packages

In [28]:
import pandas as pd
import json
import requests

## Functions

In [29]:
def getDateOutOfDatetimeToIndex(df):
    df['time'] = df['date'].apply(lambda x: str(x).split()[1])
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y %H:%M')
    df['date'] = df['date'].apply(lambda x: str(x).split()[0])
    df.set_index('date', inplace=True)
    df.sort_index(inplace=True)
    return df


In [36]:
def makeDataset(df1_raw,df2_raw, shift=False):
    df1 = df1_raw.copy()
    # 27/12/2020 10:25 -> 27-12-2020
    df1['date'] = pd.to_datetime(df1['date'], format='%d/%m/%Y %H:%M')

    df1 = getDateOutOfDatetimeToIndex(df1)


    df2 = df2_raw.copy()

    df2['profit'] = (df2['Close'] - df2['Open'])
    df2['label'] = df2['profit'].apply(lambda x: 1 if x > 0 else 0)
    df2.rename(columns={'Date':'date'},inplace=True)
    # profit will be influenced by news on same day (Shift
    if shift:
        df2['label'] = df2['label'].shift(-1)
        # remove row with nan value
        df2.dropna(inplace=True)
    else:
        pass

    df2.set_index('date', inplace=True)

    df_raw = df1.merge(on='date', right=df2[['label']], how='inner')


    df_raw = df_raw[(df_raw['time'] > '10:00') & (df_raw['time'] < '18:00')]

    df_raw['label'].value_counts()


    return df_raw

## Get data

### News data

In [31]:
url = 'https://raw.githubusercontent.com/guico3lho/TCC/main/assets/Datasets/suno/suno-petr4.json?token=GHSAT0AAAAAABYDRQ4ZASOZJOMO2OAKI4MEZAOJOKA'
r = requests.get(url)
json_data = r.json()
df_news_raw = pd.DataFrame(json_data)
df_news_raw

,topic,title,date,search_date,url,tags
0,Negócios,Petrobras (PETR4) finaliza venda do Polo Peroá...,03/08/2022 13:46,2022-08-04 00:21:19,https://www.suno.com.br/noticias/petrobras-pet...,"[Petrobras (PETR4), 3R Petroleum (RRRP3), Petr..."
1,Economia,Conselheiro da Petrobras (PETR4) sugere congel...,17/06/2022 19:52,2022-08-04 00:21:35,https://www.suno.com.br/noticias/petrobras-pet...,"[Petrobras (PETR4), Arthur Lira, Jair Bolsonar..."
2,Negócios,Radar: MRV (MRVE3) paga dividendos milionários...,20/06/2022 23:05,2022-08-04 00:21:35,https://www.suno.com.br/noticias/radar-petrobr...,"[Ibovespa, Ibovespa, Magazine Luiza (MGLU3), M..."
3,Política,Líder dos caminhoneiros sobre voucher: “Não pr...,22/06/2022 17:24,2022-08-04 00:21:35,https://www.suno.com.br/noticias/lider-caminho...,"[Petrobras (PETR4), combustíveis, diesel, Jair..."
4,Negócios,Petrobras (PETR4): Petroleiros aprovam indicat...,10/07/2022 09:40,2022-08-04 00:21:35,https://www.suno.com.br/noticias/petrobras-pet...,"[Petrobras (PETR4), greves, Petrobras (PETR4),..."
...,...,...,...,...,...,...
1425,Mercado,"Ibovespa fecha em queda de 0,35%, a 104.109,07...",28/07/2020 17:37,2022-05-03 16:22:55,https://www.suno.com.br/noticias/ibovespa-fech...,"[AES Tietê (TIET3/TIET4/TIET11), BNDES, Iboves..."
1426,Negócios,"Petrobras faz pré-pagamento de US$ 3,5 bi em l...",28/07/2020 15:00,2022-05-03 16:22:56,https://www.suno.com.br/noticias/petrobras-pre...,"[PETR3, Petrobras (PETR4)]"
1427,Negócios,Petrobras (PETR4) divulga teaser de venda de p...,28/07/2020 11:26,2022-05-03 16:22:57,https://www.suno.com.br/noticias/petrobras-pet...,"[PETR3, Petrobras (PETR4)]"
1428,Negócios,Agenda do Dia: Petrobras; Yduqs; Oi; AES Tietê...,28/07/2020 09:49,2022-05-03 16:22:57,https://www.suno.com.br/noticias/agenda-do-dia...,"[AES Tietê (TIET3/TIET4/TIET11), agenda do dia..."


### Historical data

In [32]:
# read csv
df_historical = pd.read_csv('https://raw.githubusercontent.com/guico3lho/TCC/main/assets/Datasets/historical_data/PETR4.SA_2020_2022.csv?token=GHSAT0AAAAAABYDRQ4YHUL4MN5RAT5ILM7OZAOKZTA')
df_historical

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-07-16,23.160000,23.280001,22.520000,22.719999,10.791756,69698700
1,2020-07-17,22.830000,23.030001,22.580000,22.740000,10.801254,49927600
2,2020-07-20,22.700001,22.910000,22.520000,22.740000,10.801254,48679800
3,2020-07-21,23.150000,23.600000,23.139999,23.370001,11.100499,101769600
4,2020-07-22,23.299999,23.410000,23.010000,23.049999,10.948501,43085500
...,...,...,...,...,...,...,...
359,2021-12-29,28.700001,28.900000,28.420000,28.540001,16.678885,35508400
360,2021-12-30,28.549999,28.700001,28.389999,28.450001,16.626287,43229100
361,2022-01-03,28.540001,29.219999,28.530001,29.090000,17.000307,52704700
362,2022-01-04,29.160000,29.400000,28.910000,29.200001,17.064590,51739200


### Merge datasets

In [37]:
df_raw = makeDataset(df_news_raw, df_historical)
df_raw

,topic,title,search_date,url,tags,time,label
date,,,,,,,
2020-07-16,Mercado,"Ibovespa hoje cai 1,22% em mau humor com bater...",2022-05-03 16:22:45,https://www.suno.com.br/noticias/ibovespa-fech...,"[Ibovespa, JHSF (JHSF3), Petrobras (PETR4), Va...",17:28:00,0
2020-07-21,Negócios,Petrobras (PETR4) tem queda de 3% na produção ...,2022-05-03 16:22:45,https://www.suno.com.br/noticias/petrobras-pet...,"[ANP, PETR3, Petrobras (PETR4), Petróleo]",12:32:00,1
2020-07-21,Negócios,Petrobras tem decisão favorável da Justiça em ...,2022-05-03 16:22:46,https://www.suno.com.br/noticias/petrobras-tev...,"[COFINS, PETR3, Petrobras (PETR4), PIS]",10:14:00,1
2020-07-21,Mercado,Ibovespa se descola das bolsas no exterior e c...,2022-05-03 16:22:44,https://www.suno.com.br/noticias/ibovespa-fehc...,"[Ibovespa, Paulo Guedes, Petrobras (PETR4), Qu...",17:41:00,1
2020-07-21,Mercado,"Ibovespa abre em alta de 0,6% e opera acima de...",2022-05-03 16:22:45,https://www.suno.com.br/noticias/ibovespa-ue-b...,"[Ibovespa, Petrobras (PETR4), União Europeia, ...",10:42:00,1
...,...,...,...,...,...,...,...
2021-12-27,Negócios,Petrobras (PETR4): XP diz que ação está barata...,2022-05-03 16:01:41,https://www.suno.com.br/noticias/petrobras-pet...,"[Petrobras (PETR4), XP Investimentos]",15:08:00,1
2021-12-28,Negócios,"Petrobras (PETR4) recupera R$ 6,17 bilhões em ...",2022-05-03 16:03:23,https://www.suno.com.br/noticias/petrobras-pet...,"[Operação Lava Jato, Petrobras (PETR4)]",13:22:00,0
2021-12-28,Negócios,"Perdas robustas, medo e instabilidade: relembr...",2022-05-03 16:03:23,https://www.suno.com.br/noticias/10-piores-dia...,"[Ibovespa, Petrobras (PETR4), Vale (VALE3)]",10:00:00,0


In [34]:
# 2020-07-16 -> 2022-01-06

In [38]:
df_raw['label'].value_counts()

0    281
1    204
Name: label, dtype: int64

In [ ]:
# only samples with 'time' > 10:00 and < 16:00
df_raw = df_raw[(df_raw['time'] > '10:00') & (df_raw['time'] < '18:00')]